In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import os
import matplotlib.pyplot as plt

from datasets import BiosensorDataset
from unet import UNet
from train import train_model

In [ ]:
BIO_LENGTH = 32

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
data_path = 'data_with_centers/'
dataset = BiosensorDataset(data_path, mask_type=bool, biosensor_length=BIO_LENGTH, mask_size=80)

TRAIN_SIZE = int(len(dataset)*0.8)
VAL_SIZE = len(dataset) - train

train_data, val_data = torch.utils.data.random_split(data_bool, [TRAIN_SIZE, VAL_SIZE])
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
model = UNet(n_channels=BIO_LENGTH, n_classes=1)
model = model.to(device)

In [ ]:
try:
    train_model(
        model,
        device,
        train_loader,
        val_loader,
        learning_rate=0.0001,
        epochs=3,
        checkpoint_dir=checkpoint_dir,
        amp=True,
    )
except torch.cuda.OutOfMemoryError:
    torch.cuda.empty_cache()
    print('Detected OutOfMemoryError!')